In [70]:
from typing import Literal

import numpy as np

In [71]:
def integral_image(img: np.ndarray) -> np.ndarray:
    ii = img.astype(np.int64).cumsum(axis=0).cumsum(axis=1)
    ii = np.pad(ii, ((1, 0), (1, 0)), mode="constant")

    return ii

In [72]:
def pad_image(
        img: np.ndarray,
        k: int,
        mode: Literal["valid","same"] = "valid",
        border: Literal["constant","edge","reflect"] = "constant"
) -> np.ndarray:
    if mode != "same":
        return img

    p_top = k // 2
    p_left = k // 2
    p_bottom = k - 1 - p_top
    p_right = k - 1 - p_left
    pad_spec = ((p_top, p_bottom), (p_left, p_right))

    return np.pad(img, pad_spec, mode=border)


def box_sum(
        img: np.ndarray,
        k: int,
        mode: Literal["valid","same"] = "valid",
        border: Literal["constant","edge","reflect"] = "constant"
) -> np.ndarray:
    padded_img = pad_image(img, k, mode, border)
    ii = integral_image(padded_img)

    return ii[k:, k:] - ii[:-k, k:] - ii[k:, :-k] + ii[:-k, :-k]

In [73]:
def box_mean(
    img: np.ndarray,
    k: int,
    mode: Literal["valid","same"] = "valid",
    border: Literal["constant","edge","reflect"] = "constant",
    count_normalized: bool = False,
) -> np.ndarray:
    sums = box_sum(img, k, mode=mode, border=border)

    if mode == "valid" or (mode == "same" and border in {"edge", "reflect"}) or not count_normalized:
        return sums / (k * k)

    ones = np.ones_like(img)
    counts = box_sum(ones, k, mode=mode, border=border).astype(np.float64)

    return sums / np.maximum(counts, 1.0)


In [74]:
img = np.arange(16).reshape((4, 4)) + 1

In [75]:
pad_image(img, 2, "valid", "constant")

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [76]:
box_mean(img, 2, "valid", "constant")

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [77]:
box_mean(img, 2, "valid", "constant", count_normalized=True)

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [78]:
pad_image(img, 2, "valid", "edge")

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [79]:
box_mean(img, 2, "valid", "edge")

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [80]:
box_mean(img, 2, "valid", "edge", count_normalized=True)

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [81]:
pad_image(img, 2, "valid", "reflect")

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [82]:
box_mean(img, 2, "valid", "reflect")

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [83]:
box_mean(img, 2, "valid", "reflect", count_normalized=True)

array([[ 3.5,  4.5,  5.5],
       [ 7.5,  8.5,  9.5],
       [11.5, 12.5, 13.5]])

In [84]:
pad_image(img, 2, "same", "constant")

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  5,  6,  7,  8],
       [ 0,  9, 10, 11, 12],
       [ 0, 13, 14, 15, 16]])

In [85]:
box_mean(img, 2, "same", "constant")

array([[ 0.25,  0.75,  1.25,  1.75],
       [ 1.5 ,  3.5 ,  4.5 ,  5.5 ],
       [ 3.5 ,  7.5 ,  8.5 ,  9.5 ],
       [ 5.5 , 11.5 , 12.5 , 13.5 ]])

In [86]:
box_mean(img, 2, "same", "constant", count_normalized=True)

array([[ 1. ,  1.5,  2.5,  3.5],
       [ 3. ,  3.5,  4.5,  5.5],
       [ 7. ,  7.5,  8.5,  9.5],
       [11. , 11.5, 12.5, 13.5]])

In [87]:
pad_image(img, 2, "same", "edge")

array([[ 1,  1,  2,  3,  4],
       [ 1,  1,  2,  3,  4],
       [ 5,  5,  6,  7,  8],
       [ 9,  9, 10, 11, 12],
       [13, 13, 14, 15, 16]])

In [88]:
box_mean(img, 2, "same", "edge")

array([[ 1. ,  1.5,  2.5,  3.5],
       [ 3. ,  3.5,  4.5,  5.5],
       [ 7. ,  7.5,  8.5,  9.5],
       [11. , 11.5, 12.5, 13.5]])

In [89]:
box_mean(img, 2, "same", "edge", count_normalized=True)

array([[ 1. ,  1.5,  2.5,  3.5],
       [ 3. ,  3.5,  4.5,  5.5],
       [ 7. ,  7.5,  8.5,  9.5],
       [11. , 11.5, 12.5, 13.5]])

In [90]:
pad_image(img, 2, "same", "reflect")

array([[ 6,  5,  6,  7,  8],
       [ 2,  1,  2,  3,  4],
       [ 6,  5,  6,  7,  8],
       [10,  9, 10, 11, 12],
       [14, 13, 14, 15, 16]])

In [91]:
box_mean(img, 2, "same", "reflect")

array([[ 3.5,  3.5,  4.5,  5.5],
       [ 3.5,  3.5,  4.5,  5.5],
       [ 7.5,  7.5,  8.5,  9.5],
       [11.5, 11.5, 12.5, 13.5]])

In [92]:
box_mean(img, 2, "same", "reflect", count_normalized=True)

array([[ 3.5,  3.5,  4.5,  5.5],
       [ 3.5,  3.5,  4.5,  5.5],
       [ 7.5,  7.5,  8.5,  9.5],
       [11.5, 11.5, 12.5, 13.5]])